In [1]:
import cv2 as cv
import numpy as np
import pandas as pd

In [26]:
DIFF_THRESHOLD = 0.3
PIXEL_THRESHOLD = 0.6 

In [25]:
def calculate_statistics(ndvi_array, pixel_threshold=PIXEL_THRESHOLD, diff_threshold=DIFF_THRESHOLD):
    '''
    This function generate a dictionary counting the percentage of pixels for every
    NDVI graduations (keys of the dictionary) of a numpy array made of NDVI values
    (a value for every pixel).
    This function also computes the 'diff' value, it is the percentage of pixels with
    a low NDVI (less vegetation) near every high NDVI pixels (every pixel with more
    than pixel_threshold NDVI value).
    '''
    NDVIGraduation = {
        0 : 0, # <0.1
        1 : 0, # 0.1-0.2
        2 : 0, # 0.2-0.3
        3 : 0, # 0.3-0.4
        4 : 0, # 0.4-0.5
        5 : 0, # 0.5-0.6
        6 : 0, # 0.6-0.7
        7 : 0, # 0.7-0.8
        8 : 0, # 0.8-0.9
        9 : 0, # 0.9-1.0
        'diff' : 0 # Number of pixel with contrast (Forest-Desert, Forest-Cities, Forest-Soil)
    }

    shape = np.shape(ndvi_array)
    # Map values from 0-255 to 0-1
    temp = ndvi_array / 255.0
    # Calculate the number of pixels
    nofpixel = 1.0 * shape[0] * shape[1]
    
    for i, val in enumerate(np.histogram(temp[1:-1,1:-1], bins=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])[0]):
        NDVIGraduation[i] = val/nofpixel
     
    diff_10 = np.where((temp - np.roll(temp,shift=1,axis=0)) > diff_threshold,1,0)
    diff_11 = np.where((temp - np.roll(temp,shift=1,axis=1)) > diff_threshold,1,0)
    diff_array_thr = np.where( temp>pixel_threshold, diff_10+diff_11, 0)
    
    NDVIGraduation['diff'] = (diff_array_thr[1:-1,1:-1].sum())/nofpixel

    return NDVIGraduation

In [27]:
def calculateNDVI(image):
    '''
    This function calculates the NDVI (Normalized Difference
    Vegetation Index) for each pixel of the photo and collect
    these values in "ndvi" numpy array.
    '''
    # Extract bgr values
    b, _, r = cv.split(image)
    bottom = (r.astype(float) + b.astype(float))
    # Change zeros of bottom array  
    # (to make sure to not divide by zero)
    bottom[bottom == 0] = 0.0000000000001

    # Calculate NDVI value of each pixel
    ndvi = (r.astype(float) - b) / bottom
    
    ndvi = contrast_stretch(ndvi)
    ndvi = ndvi.astype(np.uint8)
    
    return ndvi

In [28]:
def contrast_stretch(im):
    '''
    Performs a simple contrast stretch of the given image, from 5-100%.
    '''
    in_min = np.percentile(im, 5)
    in_max = np.percentile(im, 100)

    out_min = 0.0
    out_max = 255.0

    out = im - in_min
    out *= ((out_min - out_max) / (in_min - in_max))
    out += in_min

    return out

In [48]:
from pprint import pformat

csv_file = open('vegetation_new.csv', 'w')

photo_list = [42,43,60,61,74,75,85,98, 99]

for photo_counter in photo_list:
    
    print(f'Photo number {photo_counter}...')

    img = cv.imread("woodsmissionteam_img_" + str(photo_counter).zfill(3) +".png")
    
    ndvi = 255-calculateNDVI(img)

    cv.imwrite("woodsmissionteam_img_" + str(photo_counter).zfill(3) +"_NDVI.png",ndvi)

    ndvi_color = cv.applyColorMap(ndvi, cv.COLORMAP_JET)

    cv.imwrite("woodsmissionteam_img_" + str(photo_counter).zfill(3) +"_NDVI_COLOR_JET.png", ndvi_color)
    
    stats = list(calculate_statistics(ndvi).values())
    str_stats = ','.join(list(map(str, stats)))
    
    csv_file.write(f'{photo_counter}, {str_stats}')
    
    if photo_counter==75 or photo_counter==98:
        
        print(f'Photo number {photo_counter} (tagliata)...')
        
        img = cv.imread("woodsmissionteam_img_" + str(photo_counter).zfill(3) +"_tagliata.png")
        
        ndvi = 255-calculateNDVI(img)

        cv.imwrite("woodsmissionteam_img_" + str(photo_counter).zfill(3) +"_NDVI_tagliata.png",ndvi)

        ndvi_color = cv.applyColorMap(ndvi, cv.COLORMAP_JET)

        cv.imwrite("woodsmissionteam_img_" + str(photo_counter).zfill(3) +"_NDVI_COLOR_JET_tagliata.png", ndvi_color)
        
        stats = list(calculate_statistics(ndvi).values())
        
        str_stats = ','.join(list(map(str, stats)))

        csv_file.write(f'{photo_counter}, {str_stats}')
        
print('Complete')

csv_file.close()

Photo number 42...
Photo number 43...
Photo number 60...
Photo number 61...
Photo number 74...
Photo number 75...
Photo number 75 (tagliata)...
Photo number 85...
Photo number 98...
Photo number 98 (tagliata)...
Photo number 99...
Complete
